In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [11]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='utf-8')

In [12]:
texts = df.review.tolist()
sentiment = df.sentiment.tolist()

In [13]:
amount = 1000
X_train, X_test, y_train, y_test = train_test_split(texts[:amount], sentiment[:amount])

In [14]:
# clean text
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def word_tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')#fix
    clean = tokenizer.tokenize(text)
    return clean
   
def stop_words(tokenize_words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in tokenize_words:
        if w not in stopWords:
            wordsFiltered.append(w.lower())
    return wordsFiltered


def stem(text_arr):
    ps = PorterStemmer()
    stem_arr = []
    for word in text_arr:
        stem_arr.append(ps.stem(word))
    return stem_arr

def preprocess_text(text):
    #clean html 
    clean = cleanhtml(text)
    
    # tokenize and remove punctuation 
    tokenize = word_tokenize(clean)
    
    # stop words
    stop = stop_words(tokenize)
    
    # stem 
    stem_arr = stem(stop)
    
    return stem_arr


def preprocess_texts(texts):
    tmp_set = set()
    for text in texts:
        arr = preprocess_text(text) 
        tmp_set.update(arr)
    return tmp_set

print(preprocess_text(df.to_dict('records')[0]['review']))

['with', 'stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari', 'watch', 'the', 'wiz', 'watch', 'moonwalk', 'mayb', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema', 'origin', 'releas', 'some', 'subtl', 'messag', 'mj', 'feel', 'toward', 'press', 'also', 'obviou', 'messag', 'drug', 'bad', 'kay', 'visual', 'impress', 'cours', 'michael', 'jackson', 'unless', 'remot', 'like', 'mj', 'anyway', 'go', 'hate', 'find', 'bore', 'some', 'may', 'call', 'mj', 'egotist', 'consent', 'make', 'movi', 'but', 'mj', 'fan', 'would', 'say', 'made', 'fan', 'true', 'realli', 'nice', 'the', 'actual', 'featur', 'film', 'bit', 'final', 'start', '20', 'minut', 'exclud', 'smooth', 'crimin', 'sequenc', 'joe', 'pesci', 'convinc', 'psychopath', 'power', 'drug', 'lord', 'whi', 'want', 'mj', 'dead', 'bad', 'be

In [16]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# print(train_texts)
corpus = X_train

vocab_tf = preprocess_texts(corpus)

tfidf = TfidfVectorizer(stop_words = 'english', vocabulary=vocab_tf)
X = tfidf.fit_transform(corpus)
X_dense = X.todense()

# print(tfidf.get_feature_names())
X_test_transform = tfidf.transform(X_test)
X_test_transform_dense = X_test_transform.todense()

In [17]:
# KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusters = range(2,10)

for n_clusters in clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(X_dense)
    centers = clusterer.cluster_centers_

    score = silhouette_score(X_dense, preds)
    print("n clusters = {}, silhouette score is {})".format(n_clusters, score))

n clusters = 2, silhouette score is 0.002737106606338904)
n clusters = 3, silhouette score is 0.001292570041600025)
n clusters = 4, silhouette score is 0.0012257763295509033)
n clusters = 5, silhouette score is 0.0019496912820422922)
n clusters = 6, silhouette score is -0.00043158609389703454)
n clusters = 7, silhouette score is 0.0003233193644542233)
n clusters = 8, silhouette score is -0.0003621400046931758)
n clusters = 9, silhouette score is 0.001089552518658691)


In [18]:
from sklearn.cluster import AgglomerativeClustering    

clusters = range(2,15)

for n_clusters in clusters:
    clusterer = AgglomerativeClustering(n_clusters=n_clusters, linkage='average')
    preds = clusterer.fit_predict(X_dense)

    score = silhouette_score(X_dense, preds)
    print("n clusters = {}, silhouette score is {})".format(n_clusters, score))


n clusters = 2, silhouette score is 0.011538330989623735)
n clusters = 3, silhouette score is 0.00715896005739769)
n clusters = 4, silhouette score is 0.005223290192891512)
n clusters = 5, silhouette score is 0.0038835965138133825)
n clusters = 6, silhouette score is 0.003278005709045163)
n clusters = 7, silhouette score is 0.002390840197083303)
n clusters = 8, silhouette score is 0.0016568445207679472)
n clusters = 9, silhouette score is 0.0016462717331154875)
n clusters = 10, silhouette score is 0.0011130159379573355)
n clusters = 11, silhouette score is 0.0011784284873401698)
n clusters = 12, silhouette score is 0.000567130971221925)
n clusters = 13, silhouette score is 0.0008488792337937323)
n clusters = 14, silhouette score is 0.0007325382721427397)


In [19]:
max_clusterer = KMeans(n_clusters=2)
max_preds = max_clusterer.fit_predict(X_dense)
max_centers = max_clusterer.cluster_centers_
print(max_centers)
print(len(max_centers[0]))

[[ 0.00000000e+00  1.00429122e-03  2.14117761e-03 ...  3.79668117e-04
   4.06999625e-04  5.41529511e-04]
 [ 0.00000000e+00  4.52968325e-04  8.60821474e-04 ...  2.98155597e-19
   4.33680869e-19 -1.62630326e-19]]
11404


In [20]:
# vocab_tf contains vocab
# max_preds pred
# corpus - texts 
clean_texts = []
for text_idx in range(len(corpus)):
    clean_texts.append({'words': preprocess_text(corpus[text_idx]), 'ans': max_preds[text_idx]})

In [21]:
pos_amount = sum(max_preds)
neg_amount = len(max_preds) - pos_amount
print(pos_amount, neg_amount, len(max_preds))
vocab_tf_list = list(vocab_tf)
vocab_tf_list_range = range(len(vocab_tf_list))
pos_amount_words = [0 for i in vocab_tf_list_range]
neg_amount_words = [0 for i in vocab_tf_list_range]
all_amount_words = [0 for i in vocab_tf_list_range]
for vocab_word_idx in vocab_tf_list_range:
    for text in clean_texts:
        if(vocab_tf_list[vocab_word_idx] in text['words']):
            all_amount_words[vocab_word_idx] += 1
            if text['ans'] == 1:
                pos_amount_words[vocab_word_idx] += 1
            else: 
                neg_amount_words[vocab_word_idx] += 1

540 210 750


In [23]:
import math

def pmiCalc(classProb, allProb):
    if classProb == 0 or allProb == 0: 
        return 0
    return math.log10(classProb / allProb)

In [25]:
neg_pmi = []
pos_pmi = []
for idx in vocab_tf_list_range:
    pos_pmi_calc = pmiCalc(pos_amount_words[idx], all_amount_words[idx])
    if pos_pmi_calc != 0:
        pos_pmi.append({'idx': idx, 'pmi': pos_pmi_calc})  
    
    neg_pmi_calc = pmiCalc(neg_amount_words[idx], all_amount_words[idx])
    if neg_pmi_calc != 0:
        neg_pmi.append({'idx': idx, 'pmi': neg_pmi_calc})  

In [26]:
sorted_pos_pmi = sorted(pos_pmi, key = lambda x : x['pmi'], reverse = True)
sorted_neg_pmi = sorted(neg_pmi, key = lambda x : x['pmi'], reverse = True)

top_amount = 10
print(f'Top {top_amount} positive words')
for word in range(top_amount):
    print(vocab_tf_list[sorted_pos_pmi[word]['idx']])
print('------------')
print(f'Top {top_amount} nagetive words')
for word in range(top_amount):
    print(vocab_tf_list[sorted_neg_pmi[word]['idx']])

Top 10 positive words
adventur
train
everybodi
u
mouth
dad
radio
interpret
70
5
------------
Top 10 nagetive words
context
elev
fascism
importantli
festiv
devoid
merit
newcom
spanish
loneli
